In [4]:
from torch import nn
import torch
# torch.manual_seed(42)

class TwoLayerLSTM(nn.Module):
    """
    更高效的 LSTM 实现，处理时间序列数据
    """
    def __init__(self, k, hidden_dim=32, lstm_hidden_dim=64, dropout_rate=0.0):
        super(TwoLayerLSTM, self).__init__()
        self.k = k
        self.hidden_dim = hidden_dim
        self.lstm_hidden_dim = lstm_hidden_dim
        
        # LSTM 层
        self.lstm = nn.LSTM(k, lstm_hidden_dim, batch_first=True)
        
        # 全连接层
        self.fc_layers = nn.Sequential(
            nn.BatchNorm1d(lstm_hidden_dim),
            nn.Linear(lstm_hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate) if dropout_rate > 0 else nn.Identity(),
            nn.Linear(hidden_dim, 1)
        )
    
    def forward(self, x):
        # x shape: (batch_size, N, lookback, k)
        batch_size, N, lookback, k = x.shape
        
        # 重塑为 LSTM 输入: (batch_size*N, lookback, k)
        x_lstm = x.reshape(-1, lookback, k)
        
        # LSTM 处理
        lstm_out, _ = self.lstm(x_lstm)
        # 使用最后一个时间步的输出
        lstm_final = lstm_out[:, -1, :]  # shape: (batch_size*N, lstm_hidden_dim)
        
        # 全连接层处理
        output = self.fc_layers(lstm_final)  # shape: (batch_size*N, 1)
        
        # 重塑为最终输出
        output = output.view(batch_size, N)  # shape: (batch_size, N)
        
        return output

In [2]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# 在导入任何 pyepo 模块之前，先修复 HAS_GUROBI
import sys
import importlib

# 强制重新加载模块以确保修改生效
if 'pyepo.model.grb.grbmodel' in sys.modules:
    del sys.modules['pyepo.model.grb.grbmodel']
if 'pyepo.model.grb' in sys.modules:
    del sys.modules['pyepo.model.grb']

# 导入并修复
from pyepo.model.grb import grbmodel
grbmodel.HAS_GUROBI = True

# 现在导入基类
from pyepo.model.grb.grbmodel import optGrbModel


In [3]:
import numpy as np
import torch
import gurobipy as gp
from gurobipy import GRB

from pyepo.model.grb.grbmodel import optGrbModel


class MarketNeutralGrbModel(optGrbModel):
    """
    Enhanced Market Neutral Portfolio Optimization Model with Turnover Constraints
    
    This model includes:
    - Market neutral constraint (sum of weights = 1)  
    - Risk factor constraint |risk_f' x| <= risk_abs
    - Single asset position limits |x_i| <= single_abs
    - L1 norm constraint ∑|x_i| <= l1_abs
    - Quadratic risk constraint x' * cov_matrix * x <= sigma_abs
    - Optional turnover constraint ∑|x_i - w_prev_i| <= turnover
    """
    
    def __init__(
        self,
        N,
        A,
        b,
        l,
        u,
        minimize=False,
        risk_f=None,
        risk_abs=None,
        single_abs=None,
        l1_abs=None,
        cov_matrix=None,
        sigma_abs=None,
        turnover=None
    ):
        """
        Initialize the Market Neutral Optimization Model
        
        Args:
            N (int): Number of assets
            A (np.ndarray): Equality constraint matrix (1 x N)
            b (np.ndarray): Equality constraint RHS (1,)
            l (np.ndarray): Lower bounds for variables (N,)
            u (np.ndarray): Upper bounds for variables (N,)
            minimize (bool): Whether to minimize (True) or maximize (False) objective
            risk_f (np.ndarray): Risk factor vector (N,)
            risk_abs (float): Risk factor constraint bound
            single_abs (float): Single asset position limit
            l1_abs (float): L1 norm constraint bound
            cov_matrix (np.ndarray): Covariance matrix (N x N)
            sigma_abs (float): Quadratic risk constraint bound
            turnover (float): Maximum allowed turnover (optional)
        """
        # 保存所有参数
        self.N = N
        self.A = A
        self.b = b
        self.l = l
        self.u = u
        self.minimize = minimize
        self.risk_f = risk_f
        self.risk_abs = risk_abs
        self.single_abs = single_abs
        self.l1_abs = l1_abs
        self.cov_matrix = cov_matrix
        self.sigma_abs = sigma_abs
        self.turnover = turnover
        
        # 保存前期权重，默认为等权重（首次投资）
        self.w_prev = np.ones(N) / N  # Equal weights instead of zeros

        super().__init__()

    def setPrevWeights(self, w_prev):
        """
        Set previous portfolio weights for turnover constraint
        
        Args:
            w_prev (np.ndarray): Previous portfolio weights
        """
        if len(w_prev) != self.N:
            raise ValueError("Size of previous weights must match number of assets")
        
        # Check if w_prev is a PyTorch tensor
        if isinstance(w_prev, torch.Tensor):
            w_prev = w_prev.detach().cpu().numpy()
        else:
            w_prev = np.asarray(w_prev, dtype=np.float32)
            
        self.w_prev = w_prev
        
        # Update turnover constraints if they exist
        if self.turnover is not None:
            self._updateTurnoverConstraints()

    def _getModel(self):
        """
        Build the Gurobi optimization model
        
        Returns:
            tuple: (model, variables)
        """
        # 新建 Gurobi 模型
        m = gp.Model()

        # 添加原始变量 x[i]
        x = m.addVars(
            self.N,
            lb={i: float(self.l[i]) for i in range(self.N)},
            ub={i: float(self.u[i]) for i in range(self.N)},
            name="x"
        )

        # 用于 L1 约束的辅助变量 t[i]
        t = m.addVars(self.N, lb=0.0, name="t")
        
        # 用于换手率约束的辅助变量 s[i] (如果启用换手率约束)
        if self.turnover is not None:
            s = m.addVars(self.N, lb=0.0, name="s")  # s[i] >= |x[i] - w_prev[i]|

        # 设置优化方向
        m.modelSense = GRB.MINIMIZE if self.minimize else GRB.MAXIMIZE

        # ——— 1) 市场中性约束 sum A[0,i] * x[i] == b[0]
        m.addConstr(
            gp.quicksum(self.A[0, i] * x[i] for i in range(self.N)) == float(self.b[0]),
            name="eq_sum"
        )

        # ——— 2) 风险约束 |risk_f' x| <= risk_abs
        if self.risk_f is not None and self.risk_abs is not None:
            expr_r = gp.quicksum(self.risk_f[i] * x[i] for i in range(self.N))
            m.addConstr(expr_r <= float(self.risk_abs), name="risk_ub")
            m.addConstr(expr_r >= -float(self.risk_abs), name="risk_lb")

        # ——— 3) 单项绝对值约束 |x_i| <= single_abs
        if self.single_abs is not None:
            for i in range(self.N):
                m.addConstr(x[i] <=  float(self.single_abs),  name=f"single_ub_{i}")
                m.addConstr(x[i] >= -float(self.single_abs), name=f"single_lb_{i}")

        # ——— 4) L1 约束 ∑|x_i| <= l1_abs
        #   实现方式： t[i] >=  x[i], t[i] >= -x[i], 然后 ∑ t[i] <= l1_abs
        if self.l1_abs is not None:
            for i in range(self.N):
                m.addConstr(t[i] >=  x[i], name=f"t_pos_{i}")
                m.addConstr(t[i] >= -x[i], name=f"t_neg_{i}")
            m.addConstr(t.sum() <= float(self.l1_abs), name="l1_norm")

        # ——— 5) 二次型约束 x' * cov_matrix * x <= sigma_abs
        if self.cov_matrix is not None and self.sigma_abs is not None:
            x_vec = np.array([x[i] for i in range(self.N)])
            expr_q = x_vec @ self.cov_matrix @ x_vec
            m.addQConstr(expr_q <= float(self.sigma_abs), name="sigma_qc")

        # ——— 6) 换手率约束 ∑|x_i - w_prev_i| <= turnover (如果启用)
        if self.turnover is not None:
            for i in range(self.N):
                # s[i] >= x[i] - w_prev[i]
                m.addConstr(s[i] >= x[i] - float(self.w_prev[i]), name=f"turnover_pos_{i}")
                # s[i] >= -(x[i] - w_prev[i]) = w_prev[i] - x[i]
                m.addConstr(s[i] >= float(self.w_prev[i]) - x[i], name=f"turnover_neg_{i}")
            
            # 换手率总约束
            m.addConstr(s.sum() <= float(self.turnover), name="turnover_total")
            
            # 保存辅助变量以便后续更新
            self._turnover_vars = s

        return m, x
    
    def _updateTurnoverConstraints(self):
        """
        Update turnover constraints when w_prev changes
        This is called automatically when setPrevWeights is used
        """
        if self.turnover is None or not hasattr(self, '_turnover_vars'):
            return
            
        # 更新换手率约束的右侧值
        for i in range(self.N):
            # 查找并更新对应的约束
            for constr in self._model.getConstrs():
                if constr.ConstrName == f"turnover_pos_{i}":
                    # s[i] >= x[i] - w_prev[i] 更新为新的 w_prev[i]
                    # 需要移除旧约束并添加新约束
                    self._model.remove(constr)
                    self._model.addConstr(
                        self._turnover_vars[i] >= self.x[i] - float(self.w_prev[i]), 
                        name=f"turnover_pos_{i}"
                    )
                elif constr.ConstrName == f"turnover_neg_{i}":
                    # s[i] >= w_prev[i] - x[i] 更新为新的 w_prev[i] 
                    self._model.remove(constr)
                    self._model.addConstr(
                        self._turnover_vars[i] >= float(self.w_prev[i]) - self.x[i], 
                        name=f"turnover_neg_{i}"
                    )
        
        # 更新模型
        self._model.update()

    def getInfo(self):
        """
        Get model information for debugging
        
        Returns:
            dict: Model information
        """
        info = {
            'num_assets': self.N,
            'has_turnover': self.turnover is not None,
            'turnover_limit': self.turnover,
            'prev_weights': self.w_prev.tolist() if self.w_prev is not None else None,
            'constraints': {
                'risk_factor': self.risk_abs is not None,
                'single_abs': self.single_abs is not None,
                'l1_norm': self.l1_abs is not None,
                'quadratic_risk': self.sigma_abs is not None
            }
        }
        return info

    def solveSequential(self, cost_vectors):
        """
        Solve multiple periods sequentially with turnover constraints
        
        Args:
            cost_vectors (np.ndarray): Cost vectors for each period (T x N)
            
        Returns:
            tuple: (solutions, objectives) where solutions is list of T solutions
                   and objectives is list of T objective values
        """
        if cost_vectors.ndim != 2:
            raise ValueError("cost_vectors must be 2D array (T x N)")
            
        T, N = cost_vectors.shape
        if N != self.N:
            raise ValueError(f"Number of assets in cost_vectors ({N}) must match model ({self.N})")
        
        solutions = []
        objectives = []
        
        # Reset to equal weights for first period
        self.w_prev = np.ones(self.N) / self.N
        
        for t in range(T):
            # Set objective for this period
            self.setObj(cost_vectors[t])
            
            # Solve optimization problem
            sol, obj = self.solve()
            solutions.append(sol)
            objectives.append(obj)
            
            # Set current solution as previous weights for next period
            if t < T - 1 and self.turnover is not None:
                self.setPrevWeights(sol)
        
        return solutions, objectives

In [19]:
 # Example parameters
N = 335
A = np.ones((1, N))
b = np.array([1.0])
l = np.zeros(N)
u = np.ones(N) * 1e6

# Risk parameters
risk_f = np.random.randn(N)
risk_abs = 1.5
single_abs = 0.1
l1_abs = 1.0
sigma_abs = 2.5
turnover = 0.3  # 30% maximum turnover

# Covariance matrix
M = np.random.randn(N, N)
cov_matrix = M.T @ M + np.eye(N) * 1e-3

# Create model with turnover constraints
market_neutral_model = MarketNeutralGrbModel(
    N, A, b, l, u, minimize=False,
    risk_f=risk_f, risk_abs=risk_abs,
    single_abs=single_abs, l1_abs=l1_abs,
    cov_matrix=cov_matrix, sigma_abs=sigma_abs,
    turnover=None
)

# Create model with turnover constraints
market_neutral_model_testing = MarketNeutralGrbModel(
    N, A, b, l, u, minimize=False,
    risk_f=risk_f, risk_abs=risk_abs,
    single_abs=single_abs, l1_abs=l1_abs,
    cov_matrix=cov_matrix, sigma_abs=sigma_abs,
    turnover=turnover
    )

In [13]:
import numpy as np
import torch
from tqdm import tqdm

from pyepo import EPO



def sequential_regret(predmodel, optmodel, dataloader, verbose=False):
    """
    Calculate sequential regret for multi-period optimization with turnover constraints
    
    This function properly handles the sequential dependency where each period's
    optimal solution depends on the previous period's portfolio weights.
    
    Args:
        predmodel (nn.Module): Neural network for cost prediction
        optmodel (MarketNeutralGrbModel): Enhanced optimization model with turnover support
        dataloader (DataLoader): PyTorch dataloader with sequential data
        verbose (bool): Whether to print detailed progress
        
    Returns:
        float: Normalized sequential regret loss
    """
    if not isinstance(optmodel, MarketNeutralGrbModel):
        raise TypeError("optmodel must be MarketNeutralGrbModel for sequential regret")
    
    # Set model to evaluation mode
    predmodel.eval()
    
    total_regret = 0.0
    total_optsum = 0.0
    num_sequences = 0
    
    if verbose:
        print("Calculating sequential regret...")
        dataloader = tqdm(dataloader, desc="Processing sequences")
    
    for batch_data in dataloader:
        x, c, w_true, z_true = batch_data
        if next(predmodel.parameters()).is_cuda:
            x, c, w_true, z_true = x.cuda(), c.cuda(), w_true.cuda(), z_true.cuda()
        
        
        with torch.no_grad():
            if predmodel.training:
                predmodel.eval()
            pred_costs = predmodel(x).to("cpu").detach().numpy()  # (batch_size, num_assets) 
            
            # Get true costs and solutions for this sequence
            true_costs = c.to("cpu").detach().numpy()  # (batch_size, num_assets) 
            true_solutions = w_true.to("cpu").detach().numpy()  # (batch_size, num_assets) 
            true_objectives = z_true.to("cpu").detach().numpy()  # (batch_size, 1) 
            
            # Calculate sequential regret for this sequence
            seq_regret, seq_optsum = _calculate_sequence_regret(
                pred_costs, true_costs, true_solutions, true_objectives, optmodel
            )
            
            total_regret += seq_regret
            total_optsum += seq_optsum
            num_sequences += 1
    
    # Turn back to train mode
    predmodel.train()
    
    # Normalized regret
    normalized_regret = total_regret / (total_optsum + 1e-7)
    
    if verbose:
        print(f"Processed {num_sequences} sequences")
        print(f"Total regret: {total_regret:.6f}")
        print(f"Total optimal sum: {total_optsum:.6f}")
        print(f"Normalized regret: {normalized_regret:.6f}")
    
    return normalized_regret


def _calculate_sequence_regret(pred_costs, true_costs, true_solutions, true_objectives, optmodel):
    """
    Calculate regret for a single sequence with sequential dependencies
    
    Args:
        pred_costs (np.ndarray): Predicted costs (T, N)
        true_costs (np.ndarray): True costs (T, N)
        true_solutions (np.ndarray): True optimal solutions (T, N)
        true_objectives (np.ndarray): True optimal objectives (T, 1)
        optmodel (MarketNeutralGrbModel): Optimization model
        
    Returns:
        tuple: (sequence_regret, sequence_optsum)
    """
    seq_len, num_assets = pred_costs.shape
    sequence_regret = 0.0
    sequence_optsum = 0.0
    
    # Reset model to equal weights for first period
    optmodel.w_prev = np.ones(num_assets) / num_assets
    
    for t in range(seq_len):
        try:
            # Set predicted costs and solve
            optmodel.setObj(pred_costs[t])
            pred_solution, _ = optmodel.solve()
            
            # Convert to numpy if needed
            if isinstance(pred_solution, torch.Tensor):
                pred_solution = pred_solution.detach().cpu().numpy()
            
            # Calculate objective value using true costs
            pred_obj_with_true_costs = np.dot(pred_solution, true_costs[t])
            # Set predicted costs and solve
            optmodel.setObj(true_costs[t])
            _, true_obj = optmodel.solve()
            
            
            # Calculate regret for this period
            if optmodel.modelSense == EPO.MINIMIZE:
                period_regret = pred_obj_with_true_costs - true_obj
            elif optmodel.modelSense == EPO.MAXIMIZE:
                period_regret = true_obj - pred_obj_with_true_costs
            else:
                raise ValueError("Invalid modelSense")
            
            sequence_regret += period_regret
            sequence_optsum += abs(true_obj)
            
            # Set current predicted solution as previous weights for next period
            if t < seq_len - 1 and optmodel.turnover is not None:
                optmodel.setPrevWeights(pred_solution)
                
        except Exception as e:
            print(f"Error in period {t}: {str(e)}")
            # Use true solution as fallback and continue
            if optmodel.turnover is not None and t < seq_len - 1:
                optmodel.setPrevWeights(true_solutions[t])
    
    return sequence_regret, sequence_optsum

In [14]:
# 加载数据集

from pyepo.data.dataset import optDataset
import numpy as np

def load_dataset_dict(filename):
    """从文件加载数据集字典"""
    data = np.load(filename, allow_pickle=True)
    return {
        'feats': data['feats'],
        'costs': data['costs'],
        'sols': data['sols'],
        'objs': data['objs'],
        'lookback': data['lookback'].item(),
        'padding_method': str(data['padding_method'])
    }

# 从字典创建数据集
def create_dataset_from_dict(dataset_dict, model):
    """从字典创建optDataset实例，并载入预计算的sols和objs"""
    dataset = optDataset(
        model=model,
        feats=dataset_dict['feats'],
        costs=dataset_dict['costs'],
        lookback=dataset_dict['lookback'],
        padding_method=dataset_dict['padding_method'],
        precomputed =True
    )
    
    # 用预计算的解替换
    dataset.sols = dataset_dict['sols']
    dataset.objs = dataset_dict['objs']
    
    return dataset


# 之后使用时，可以直接加载而无需重新计算
loaded_dict = load_dataset_dict('large_market_neutral_dataset.npz')
dataset = create_dataset_from_dict(loaded_dict, market_neutral_model)




>>> Precomputed mode enabled. Skipping time series processing and solution computation.


In [15]:
from sklearn.model_selection import train_test_split



# 获取索引并进行分割
indices = list(range(100))
split_point = int(len(indices) * 0.8)
train_indices = indices[:split_point]
test_indices = indices[split_point:]

# 创建 Subset 类
from torch.utils.data import Subset

dataset_train = Subset(dataset, train_indices)
dataset_test = Subset(dataset, test_indices)

# 现在 dataset_train 和 dataset_test 都是 optDataset 对象
print(f"Dataset shape: {dataset_train.dataset.feats.shape}")
print(f"Training size: {len(dataset_train)}")

print(f"Test size: {len(dataset_test)}")


#############################################################################
# DATA LOADERS WITH PREFETCHING
#############################################################################
import torch
from torch.utils.data import DataLoader
# Find optimal batch size based on GPU memory
# Start with a reasonable default
batch_size = 8  

generator = torch.Generator()
generator.manual_seed(42)

# Configure data loaders with prefetching
loader_train = DataLoader(
    dataset_train, 
    batch_size=batch_size, 
    shuffle=True,
    generator=generator # 固定data loader的生成器 - 用于对比MLP & MLP_Forloop
    # pin_memory=True,  # Speed up host to GPU transfers
    # num_workers=2,    # Prefetch in parallel
    # persistent_workers=True  # Keep workers alive between epochs
)

loader_test = DataLoader(
    dataset_test, 
    batch_size=batch_size, 
    shuffle=False,
    # pin_memory=True,
    # num_workers=1
)

# move loader to device for logging regret
def device_loader(loader, device):
    for batch in loader:
        x, c, w, z = batch
        yield x.to(device), c.to(device), w.to(device), z.to(device)



Dataset shape: (131010, 335, 5, 6)
Training size: 80
Test size: 20


In [16]:
#############################################################################
# TRAINING WITH MIXED PRECISION
#############################################################################
import time
from torch.amp import GradScaler, autocast
import pyepo

def trainModel(model, loss_func, method_name, num_epochs=5, lr=1e-3):
    """
    Enhanced training function with:
    - Mixed precision for faster GPU training
    - Learning rate scheduling
    - Progress bars
    - Detailed logging
    - Memory-efficient tensor handling
    """
    # Set up optimizer with weight decay for regularization
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    
    # Learning rate scheduler for better convergence
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=1
    )
    
    # Enable mixed precision training
    scaler = GradScaler(enabled=(device.type in ["cuda", "mps"]))

    # Set model to training mode
    model.train()
    
    # Initialize logs
    loss_log = []
    # evaluate loss on whole test data
    loss_log_regret = [ sequential_regret(model, market_neutral_model_testing, device_loader(loader_test, device))]
    print(f"Initial regret: {loss_log_regret[0]*100:.4f}%")
    
    # Initialize elapsed time tracking
    training_start = time.time()
    total_elapsed = 0
    
    # Verbosity control - set to false for production
    debug_mode = False
    log_interval = 10  # Log every N batches
    
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        epoch_start = time.time()
        
        # Progress bar for this epoch
        progress_bar = tqdm(loader_train, desc=f"Epoch {epoch+1}/{num_epochs}")
        
        for i, data in enumerate(progress_bar):
            x, c, w, z = data
            
            # Move data to GPU (once, not in every batch)
            x, c, w, z = x.to(device), c.to(device), w.to(device), z.to(device)
            
            # Record batch start time for accurate timing
            batch_start = time.time()
            
            # Clear gradients for each batch
            optimizer.zero_grad()
            
            # Use mixed precision where appropriate
            with autocast(device_type=device.type, enabled=(device.type in ["cuda", "mps"])):
                # Forward pass
                cp = model(x)
                
                # Compute loss based on method
                if method_name == "spo+":
                    loss = loss_func(cp, c, w, z)
                elif method_name in ["ptb", "pfy", "imle", "aimle", "nce", "cmap"]:
                    loss = loss_func(cp, w)
                elif method_name in ["dbb", "nid"]:
                    loss = loss_func(cp, c, z)
                elif method_name in ["pg", "ltr"]:
                    loss = loss_func(cp, c)
            
            # Backward pass with mixed precision handling
            if scaler is not None:
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                optimizer.step()
            
            # Track batch elapsed time
            batch_elapsed = time.time() - batch_start
            total_elapsed += batch_elapsed
            
            # Update loss tracking
            current_loss = loss.item()
            epoch_loss += current_loss
            loss_log.append(current_loss)
            
            # Update progress bar
            progress_bar.set_postfix({
                'loss': f"{current_loss:.4f}", 
                'batch time': f"{batch_elapsed:.4f}s"
            })
            
            # Debug logging (limited to avoid overwhelming output)
            if debug_mode and i % log_interval == 0:
                print(f"\n[Debug] Batch {i} stats:")
                print(f"Loss: {current_loss:.6f}")
                print(f"Pred shape: {cp.shape}, values: {cp[0,:5].detach().cpu().numpy()}")
                
                # Monitor memory usage
                if device.type == 'cuda':
                    mem_allocated = torch.cuda.memory_allocated() / 1024**2
                    mem_reserved = torch.cuda.memory_reserved() / 1024**2
                    print(f"GPU Memory: {mem_allocated:.1f}MB allocated, {mem_reserved:.1f}MB reserved")
        
        # Compute regret on test set after each epoch
        with torch.no_grad():
            model.eval()  # Set model to evaluation mode
            regret = sequential_regret(model, market_neutral_model_testing, device_loader(loader_test, device))
            model.train()  # Set back to training mode
            loss_log_regret.append(regret)
        
        # Update learning rate scheduler
        scheduler.step(epoch_loss)
        
        # End of epoch reporting
        epoch_time = time.time() - epoch_start
        print(f"Epoch {epoch+1}: Loss={epoch_loss/len(loader_train):.6f}, "
              f"Regret={regret*100:.4f}%, Time={epoch_time:.2f}s")
    
    # Report total training time
    total_training_time = time.time() - training_start
    print(f"Total training time: {total_training_time:.2f}s, "
          f"Effective computation time: {total_elapsed:.2f}s")
    
    return loss_log, loss_log_regret

#############################################################################
# TRAINING EXECUTION 
#############################################################################
print("\nInitializing SPO+ loss function...")
spop = pyepo.func.SPOPlus(market_neutral_model)


Initializing SPO+ loss function...
Num of cores: 1


In [17]:
#############################################################################
# IMPORTS AND SETUP
#############################################################################
# Set random seed for reproducibility
import random
random.seed(42)
import numpy as np
np.random.seed(42)
import torch
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True  # Makes training more deterministic
torch.backends.cudnn.benchmark = False     # Can speed up training when input sizes don't change

# Standard imports
import pandas as pd
import numpy as np
import polars as pl
from datetime import datetime, timedelta
import time
import os

from tqdm.auto import tqdm  # Progress bar


# Check and set up GPU (support MPS on Mac)
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

# Print extra info
if device.type == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    torch.cuda.empty_cache()
elif device.type == "mps":
    print("Using Apple Silicon GPU via Metal Performance Shaders (MPS)")


# ##### 强制使用CPU
# # os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# device = torch.device("cpu")
# # Force JAX to use the CPU backend only (avoids CUDA OOM in JAX)
# os.environ['JAX_PLATFORMS'] = 'cpu'

Using device: mps
Using Apple Silicon GPU via Metal Performance Shaders (MPS)


In [20]:
lstm_efficient = TwoLayerLSTM(k=6, hidden_dim=32, lstm_hidden_dim=64, dropout_rate=0.0).to(device)
print("\nStarting model training...")
loss_log, loss_log_regret = trainModel(
    lstm_efficient, 
    loss_func=spop, 
    method_name="spo+",
    num_epochs=5,  # Increased for better convergence
    lr=1e-3        # Adjusted learning rate
)


Starting model training...
Initial regret: 364.1340%


Epoch 1/5:   0%|          | 0/10 [00:00<?, ?it/s]

ValueError: Size of cost vector cannot match vars.

In [11]:
#############################################################################
# VISUALIZATION
#############################################################################
from matplotlib import pyplot as plt

def visLearningCurve(loss_log, loss_log_regret):
    """Enhanced visualization with smoother curves and more information"""
    # Create figure and subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,4))
    
    # Plot training loss with smoothing for readability
    n_points = len(loss_log)
    
    # Apply smoothing for large datasets
    if n_points > 100:
        window_size = max(10, n_points // 50)
        smoothed_loss = np.convolve(loss_log, np.ones(window_size)/window_size, mode='valid')
        x_axis = np.arange(len(smoothed_loss))
        ax1.plot(x_axis, smoothed_loss, color="c", lw=2, label=f"Smoothed (window={window_size})")
        # Also plot the raw data with transparency
        ax1.plot(loss_log, color="c", lw=0.5, alpha=0.3, label="Raw")
        ax1.legend()
    else:
        # For smaller datasets, just plot the raw data
        ax1.plot(loss_log, color="c", lw=2)
    
    ax1.tick_params(axis="both", which="major", labelsize=12)
    ax1.set_xlabel("Iterations", fontsize=16)
    ax1.set_ylabel("Loss", fontsize=16)
    ax1.set_title("Learning Curve on Training Set", fontsize=16)
    ax1.grid(True, linestyle='--', alpha=0.7)
    
    # Draw plot for regret on test set
    epochs = np.arange(len(loss_log_regret))
    ax2.plot(epochs, [r*100 for r in loss_log_regret], 
             color="royalblue", marker='o', ls="-", alpha=0.8, lw=2)
    
    ax2.tick_params(axis="both", which="major", labelsize=12)
    ax2.set_ylim(0, max(50, max([r*100 for r in loss_log_regret])*1.1))  # Dynamic y-limit
    ax2.set_xlabel("Epochs", fontsize=16)
    ax2.set_ylabel("Regret (%)", fontsize=16)
    ax2.set_title("Learning Curve on Test Set", fontsize=16)
    ax2.grid(True, linestyle='--', alpha=0.7)
    
    # Add values to points
    for i, r in enumerate(loss_log_regret):
        ax2.annotate(f"{r*100:.2f}%", 
                     (i, r*100),
                     textcoords="offset points", 
                     xytext=(0,10), 
                     ha='center')
    
    plt.tight_layout()
    # plt.savefig('learning_curves.png', dpi=300, bbox_inches='tight')
    # print("Saved learning curves to 'learning_curves.png'")
    plt.show()

print("\nVisualizing learning curves...")
visLearningCurve(loss_log, loss_log_regret)
visLearningCurve(loss_log_fl, loss_log_regret_fl)


Visualizing learning curves...


NameError: name 'loss_log' is not defined